# Chapitre 4 : Validation de modèles

Email : <a href='mailto:madani.a@ucd.ac.ma'>madani.a@ucd.ac.ma</a>
<img src='images/robot.png'>

## 1. Introduction

<p>
Il est toujours nécessaire de valider la stabilité de votre modèle d'apprentissage automatique. Cela veut dire qu'il ne suffit pas d'adapter le modèle à vos données d'entraînement et espérer qu'il fonctionnerait avec précision pour les données réelles qu'il n'a jamais vues auparavant. 
</p>
<p>
Dans ce chapitre nous allons aborder le principe de validation de modèles dans le contexte de machine learning. Après une petite description des méthodes de validation les plus populaires, nous allons présenter des exemples d'implémentation en utilisant le langage Python.
</p>

## 2. Validation

<p>
Le processus consistant à déterminer si les résultats numériques quantifiant les relations entre les variables sont acceptables en tant que descriptions des données est appelé validation.

Dans ce processus, une estimation numérique de la différence entre les réponses prédites et les réponses originales est effectuée. Cette estimation est appelée erreur d'apprentissage. Elle nous permet d'avoir une idée sur la qualité de notre modèle. 
</p>

<p>
Plusieurs modèles de validation utilisés dans le contexte de machine learning sont proposés, mais nous n'allons aborder que quelques unes. Pour plus d'informations sur d'autres modèles, consultez <a href="http://scikit-learn.org/stable/modules/cross_validation.html">ce lien </a>
</p>
<p>
Dans la suite de ce chapitre, nous allons tester quelques approches de validation appliqués à l'algorithme KNN en utilisant le dataset Iris. On peut utiliser n'importe quel autre algorithme (SVM, Regression lineaire, ...).
</p>


## 3. Overfitting vs Underfitting

Dans le contexte de machine learning, nous divisons généralement nos données en deux sous-ensembles : un sous-ensemble pour l'entraînement du modèle et un sous-ensemble pour les tests (et parfois en trois sous-ensembles pour l'entraînement, la validation et les tests). Nous commençons d'abord par adapter notre modèle sur les données d'entraînement pour faire, ensuite, des prédictions sur les données de test. Lorsque nous faisons cela, une chose peut se produire : soit on aura un <strong>Overfitting (sur-apprentissage)</strong> ou un <strong>Underfitting (sous-apprentissage)</strong>. Nous essayons d'éviter ceci, car ils affectent la prédiction de notre modèle - nous pourrions utiliser un modèle moins précis et / ou non généralisé (ce qui signifie que vous ne pouvez pas généraliser vos prédictions sur d'autres données). Voyons ce que signifient Overfitting et Underfitting :

### 3.1 Overfitting

<p>Le sur-apprentissage signifie que le modèle que nous avons formé s'est entraîné «trop bien» et qu'il s'adapte très bien au dataset d'entraînement. Cela survient généralement lorsque le modèle est trop complexe, c'est-à-dire qu'il y a trop de caractéristiques (variables ou features en anglais) par rapport au nombre d'observations. Ce modèle sera très précis sur les données d'entraînement mais sera probablement très inexact sur les données non entraînées ou de nouvelles données. C'est pour cela que ce modèle n'est pas généralisé.
</p>
<p>Fondamentalement, lorsque cela se produit, le modèle intègre le «bruit» dans les données d'apprentissage au lieu des relations réelles entre les variables dans les données. Evidemment, ce bruit ne fait partie d'aucun nouvel ensemble de données, et ne peut pas lui être appliqué.
</p>

### 3.2 Underfitting

<p>
Contrairement à l'Overfitting un modèle peut être <i>Underfitted</i>. Cela signifie que le modèle ne correspond pas aux données d'apprentissage et qu'il lui manque des tendances dans les données. Cela signifie également que le modèle ne peut pas être généralisé à de nouvelles données. Comme vous l'avez probablement deviné (ou compris!), Ceci est généralement le résultat d'un modèle très simple (pas assez de caractéristiques, features). Cela peut aussi arriver lorsque, par exemple, nous ajustons un modèle linéaire (comme la régression linéaire) à des données qui ne sont pas linéaires.
</p>
<p>
<img src="images/under_overfitting.png">
<center>Un exemple de l'Overfitting, Underfitting et d'un modèle «juste»</center>
</p>
<p>
Il est à noter que l'Underfitting n'est pas aussi répandu que l'Overfitting. Néanmoins, nous voulons éviter ces deux problèmes dans l'analyse des données. Vous pourriez dire que nous essayons de trouver le juste milieu entre l'Underfitting et l'Overfitting de notre modèle. Comme nous allons le voir par la suite, les méthodes <strong>train/test split</strong> et <strong>Cross-Validation</strong> permettent d'éviter l'Overfitting plus que l'Underfitting.
</p>

Nous allons commencer par importer le dataset Iris que nous allons utiliser dans les exemples de ce chapitre

In [3]:
# Importer load_iris() permettant de charger le dataset Iris
from sklearn.datasets import load_iris
# Charger le dataset Iris
iris = load_iris()
# X : données
X = iris.data
# y : classes
y = iris.target
#Vérifier la structure de X et y
print(X.shape, y.shape)

(150, 4) (150,)


## 4. Exemples de modèles de validation

### 4.1 Entraîner et tester sur le Dataset entier

Cette procédure, appelée aussi approche <strong>naîve</strong>, consiste à :
<ul>
<li>Entraîner le modèle sur le dataset en entier
<li>Tester le modèle sur le même dataset, et évaluer à quel point nous avons bien comparé les valeurs de classess prédites avec les vraies valeurs de réponse.
</ul>

Commençons par choisir un modèle et ses hyperparamètres. Dans notre exemple, nous allons utiliser le classificateur k-neighbors avec n_neighbors = 1.

In [9]:
from sklearn.neighbors import KNeighborsClassifier
# clf pour classifieur. Un classifieur est un modèle, tels que KNN, SVC, ...
clf = KNeighborsClassifier(n_neighbors=1)

Ensuite, nous entraînons le modèle et nous l'utilisons pour prédire les classes des données que nous connaissons déjà :

In [10]:
# Entraînement du modèle
clf.fit(X, y)
# y_pred : y prédite
y_pred = clf.predict(X)

Enfin, nous calculons la fraction des points correctement étiquetés :

In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_pred)

1.0

Comme on peut le voir, nous avons obtenu un score de précision égal 1,0, ce qui indique que 100% des points étaient correctement étiquetés par notre modèle! Mais est-ce vraiment la mesure de précision attendue ? Avons-nous vraiment aboutit à un modèle correct à 100% ?

Comme vous l'avez peut-être compris, la réponse est non. En fait, cette approche contient un défaut : il entraîne et évalue le modèle sur les mêmes données.

En conclusion, le fait d'apprendre les paramètres d'une fonction de prédiction et la tester sur les mêmes données est une erreur méthodologique : un modèle qui répèterait simplement les classes des échantillons que l'on vient de voir aurait un score parfait mais ne pourrait prédire quoi que ce soit d'utile sur des données non vues. Cette situation est appelée <strong>Overfitting</strong> (surapprentissage). Pour éviter l'Overfitting, il est courant lors de l'exécution d'une expérience de machine learning (supervisée) de conserver une partie des données pour l'entraînement et une autre partie pour le test.

### 4.2 Méthode Holdout (Train/test split)

Pour remèdier aux inconvénients de la méthode précédente, le principe de la méthode <strong>holdout</strong> consiste à supprimer une partie des données d'apprentissage et à l'utiliser pour obtenir des prédictions du modèle entraîné sur le reste des données.
</p>
<p>
L'ensemble d'apprentissage (l'ensemble d'entraînement) contient une sortie connue et le modèle apprend sur ces données afin d'être généralisé à d'autres données plus tard. Nous avons besoin, aussi, de l'ensemble de données de test afin de tester la prédiction de notre modèle sur ce sous-ensemble.
</p>
<img src="images/train_test_split1.png">
<p>
L'estimation de l'erreur indique ensuite comment notre modèle se comporte sur des données non vues auparavant. C'est un type simple de technique de validation croisée, également connu sous le nom de méthode <strong>train/test split</strong>.
</p>

Voyons maintenant comment faire cela en Python. Nous utilisons la bibliothèque <a href="http://scikit-learn.org/stable/index.html">Scikit-Learn</a> et plus particulièrement la méthode <a href="http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html">train_test_split</a>.

In [13]:
#Importer la fonction train_test_split
from sklearn.model_selection import train_test_split
# Diviser les données d'entraînement en deux sous ensembles Train et Test ayant 50% chacun
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0,train_size=0.8)
# Entraîner le modèle sur l'ensemble d'entraînement
clf.fit(X_train, y_train)
# evaluer le modèle sur l'ensemble de test
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

1.0

<p>
Nous avons utilisé la fonction <strong>train_test_split</strong> pour faire la division (le split). Le test_size = 0.5 à l'intérieur de la fonction indique le pourcentage de données à conserver pour le test. C'est généralement autour de 80/20 ou 70/30.
</p>
<p>
Comme on peut le voir ici, nous avons obtenu un résultat plus raisonnable.
</p>

### 4.3 Cross-Validation (Validation croisée)

#### 4.3.1 Two-Folds

L'un des inconvénients de l'utilisation de la méthode holdout pour la validation de modèles est que nous avons retiré une partie de nos données de l'ensemble de données destinées à l'entraînement du modèle. Dans l'exemple du cas précédent, la moitié de l'ensemble de données ne va pas contribuer à l'entraînement du modèle! Ce n'est pas optimal et peut causer des problèmes, en particulier si l'ensemble initial de données d'entraînement est petit.

Une façon de remèdier à ce problème est d'utiliser <strong>Cross-Validation</strong> (la validation croisée). L'idée est de construire plusieurs sous-ensemble (Folds) où chaque sous-ensemble de données est utilisé à la fois comme un ensemble d'apprentissage et comme un ensemble de validation. Visuellement, cela pourrait ressembler à la figure suivante.

<img src="images/two-folds.png">

Ici, nous procédons à deux tests de validation, en utilisant alternativement chaque moitié des données comme dans le cas de la méthode holdout. En utilisant les données importées auparavant, nous pourrions l'implémenter comme ceci :

In [15]:
#Importer la fonction train_test_split
from sklearn.cross_validation import train_test_split
# Diviser les données d'entraînement en deux sous ensembles Fold1 et Fold2 ayant 50% chacun
X1, X2, y1, y2 = train_test_split(X, y, random_state=0,train_size=0.5)
# Entraîner le modèle sur le premier Fold
clf.fit(X1, y1)
# evaluer le modèle sur le deuxième Fold
y_pred2 = clf.predict(X2)
# Entraîner le modèle sur le deuxième Fold
clf.fit(X2, y2)
# evaluer le modèle sur le premier Fold
y_pred1 = clf.predict(X1)
#Afficher la précision
score1 = accuracy_score(y1, y_pred1)
score2 = accuracy_score(y2, y_pred2)
score = (score1 + score2)/2
print("Score 1 :", score1, "Score 2 :", score2, " Score final :",score)


Score 1 : 0.96 Score 2 : 0.906666666667  Score final : 0.933333333333


Nous obtenons deux scores de précision, que nous pourrions combiner (par exemple, en prenant la moyenne) pour obtenir une meilleure mesure de la performance du modèle global. Cette forme particulière de validation croisée est une double validation croisée dans laquelle nous avons divisé les données en deux ensembles et utilisés chacun à son tour comme un ensemble de validation.

#### 4.3.2 K-Folds

Nous pourrions développer l'idée précédente pour utiliser encore plus de tests, et plus de split dans les données. Par exemple, la Figure suivante est une représentation visuelle de la validation croisée contenant k folds (sous-ensembles).

<img src="images/k-folds.png">

<p>
Dans l'exemple suivant, nous divisons les données en 5 groupes, et nous utilisons chacun d'eux à tour de rôle pour évaluer le modèle sur l'autre 4/5 des données.
</p>
<p>Ce serait très fastidieux de l'implémenter manuellement. Nous pouvons utiliser la fonction <strong>cross_val_score</strong> de Scikit-Learn pour le faire succinctement :
</p>

In [16]:
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(clf, X, y, cv=5)
print("Tous les scores :", scores)
print("Le score final :", scores.mean())

Tous les scores : [ 0.96666667  0.96666667  0.93333333  0.93333333  1.        ]
Le score final : 0.96


La répétition de la validation entre les différents sous-ensembles de données nous donne une meilleure idée de la performance de l'algorithme.

#### 4.3.3 Leave-One-Out-Cross-Validation (LOOCV)

Scikit-Learn met en œuvre un certain nombre de schémas de validation croisée qui sont utiles dans des situations particulières. Ces schémas sont implémentés via des itérateurs dans le module cross_validation. Par exemple, nous pourrions vouloir aller au cas extrême où notre nombre de splits est égal au nombre d'exemples du dataset ; c'est-à-dire que nous entraînons notre modèle sur toutes les données sauf une à chaque itération. Ce type de validation croisée peut être utilisé comme suit:

In [17]:
from sklearn.cross_validation import LeaveOneOut
scores = cross_val_score(clf, X, y, cv=LeaveOneOut(len(X)))
print("Tous les score :",scores)
print("Score final :",scores.mean())


Tous les score : [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  1.
  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.]
Score final : 0.96


Étant donné que nous avons 150 échantillons, la validation croisée «Leave One Out Cross Validation» donne des résultats pour 150 tets, et le score indique une prédiction réussie (1.0) ou infructueuse (0.0). La moyenne de ceux-ci nous donne une estimation du taux d'erreur.